# Chapter 3 Function
## 19. 不要把函数返回的多个数值返回到三个以上的变量

In [1]:
# 有多个返回值时，用带‘*’的表达式接受那些没有被普通变量捕获的值（元组类型）
def get_avg_retion(numbers: list):
    average = sum(numbers)/len(numbers)
    scaled = [x / average for x in numbers]
    scaled.sort(reverse=True)
    return scaled
length = [63, 73, 72, 60, 67, 66, 71, 62, 72, 70]
longest, *middle, shortest = get_avg_retion(length)
print(f'longest : {longest},\n,middle:{middle}, \nshortest:{shortest}')

longest : 1.0798816568047338,
,middle:[1.0650887573964498, 1.0650887573964498, 1.0502958579881658, 1.0355029585798818, 0.9911242603550297, 0.9763313609467457, 0.9319526627218936, 0.9171597633136096], 
shortest:0.8875739644970415


## 20. 遇到意外情况时应该抛出异常，不要返回None
返回None会和0值冲突

In [2]:
def careful_divide(a:float,b: float) -> float:
    """Divides a by b

    Raises:
    ValueError: when inputs cannot be divide
    """
    try:
        return a / b
    except ZeroDivisionError:
        raise ValueError('Invalid inputs')

In [3]:
try:
    result = careful_divide(5,2)
except ValueError:
    print('非法输出')
else:
    print(f'result is {result}')

result is 2.5


## 21. 了解如何在闭包里使用外围作用域的变量
避免函数中的局部变量污染外部模块，函数内作用域若有该变量则直接赋值，若无则以此函数范围作作用域创建变量

In [8]:
def sort_priority(values:list, group: list):
    def helper(x):
        if x in group:
            return (0, x)
        return (1, x)
    values.sort(key=helper)

In [9]:
numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = [2, 3, 5, 7]
sort_priority(numbers, group)
print(numbers)

[2, 3, 5, 7, 1, 4, 6, 8]


found的作用域在闭包函数内，赋值在外部无效，若想实现需要使用nonlocal声明

In [10]:
def sort_priority(values:list, group: list):
    found = False
    def helper(x):
        if x in group:
            return (0, x)
        found = True
        return (1, x)
    values.sort(key=helper)
    return found

In [12]:
numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = [2, 3, 5, 7]
f = sort_priority(numbers, group)
print(numbers,f)

[2, 3, 5, 7, 1, 4, 6, 8] False


In [13]:
def sort_priority(values:list, group: list):
    found = False
    def helper(x):
        nonlocal found
        if x in group:
            return (0, x)
        found = True
        return (1, x)
    values.sort(key=helper)
    return found
numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = [2, 3, 5, 7]
f = sort_priority(numbers, group)
print(numbers,f)

[2, 3, 5, 7, 1, 4, 6, 8] True


还可以使用__call__方法定义类的调用触发函数，免去闭包

In [16]:
class Sorted():
    def __init__(self, group):
        self.group = group
        self.found = False

    def __call__(self, x):
        if x in self.group:
            self.found = True
            return (0, x)
        return (1, x)

In [17]:
sortd = Sorted(group)
numbers.sort(key=sortd)
print(sortd.found)

True


## 22. 用数量可变的位置参数给函数设计清晰的参数列表

In [1]:
def log(message, values):
    if not  values:
        print(message)
    else:
        values_str = ','.join(str(x) for x in values)
        print(f'{message} : {values_str}')

In [2]:
log('My numbers are', [1, 2])
log('Hi there', [])

My numbers are : 1,2
Hi there


此时第二个参数如果不传会报错

In [3]:
log('Hi there')

TypeError: log() missing 1 required positional argument: 'values'

使用'*’在函数的位置参数中，这样只需要提供不带*参数，并不会报错

In [4]:
def log1(message, *values):  # 将所有输入的数值pack到values中
    if not  values:
        print(message)
    else:
        values_str = ','.join(str(x) for x in values)
        print(f'{message} : {values_str}')

In [9]:
log1('My numbers are', 1, 2)

My numbers are : 1,2


In [15]:
log1('My numbers are', [1, 2])
# 此时pack了一个tuple，及([1, 2]),如果想给list进行传入，则需要继续使用‘*’
log1('My numbers are', *[1, 2])

My numbers are : [1, 2]
My numbers are : 1,2


In [12]:
log1('Hi there')

Hi there


**使用‘*’作为可变的位置参数时，需要注意传入的数据大小、类型。**
如果将*加给迭代器，那么程序必须先把生成器中的元素迭代完才能继续向下执行。

In [3]:
def my_generator():
    for i in range(10):
        yield i
def my_func(*args):
    print(args)
it = my_generator()
my_func(*it)  # 先将my_generator整个迭代，再传入元组’（）‘中

(0, 1, 2, 3, 4, 5, 6, 7, 8, 9)


- *args适合处理输入值不太多，且数量可以预测的情况，最好是多个字面值或变量名
- 对于此类函数，在进行更新维护时要添加新参数时，之前写过的调用代码就要整个更新
例如如果在参数列表开头加入参数，就要更新整个代码

In [4]:
def log2(sequence, message, *values):  # 将所有输入的数值pack到values中
    if not  values:
        print(message)
    else:
        values_str = ','.join(str(x) for x in values)
        print(f'{message} : {values_str}')

In [6]:
log2(1, 'Favourite', 7, 33)
log2(1, 'HI there')
log2('Favourite numbers', 7,33)  #这种写法不会报错但并不满足函数的功能需求，排查起来很困难

Favourite : 7,33
HI there
7 : 33


- *后跟迭代器等大型数据结构有内存爆炸的可能
- 给*args的函数调用时，最后指定对应的参数关键字，避免传参混乱
- *args最好不用于特别复杂的数据拆解，适用于变量输出

## 23.用关键字参数便是可选的行为

In [8]:
def remainder(number, divisor):
    return number % divisor
assert remainder(20, 7) == 6

In [10]:
# 此时可以将位置传参、关键字传参混用
remainder(20, 7)
remainder(20, divisor=7)
remainder(number=20, divisor=7)
remainder(divisor=7, number=20)

6

In [13]:
# 需要注意的是，混合传入是不可以将关键字放在开头
remainder(number=20, 7)

SyntaxError: positional argument follows keyword argument (736591450.py, line 2)

In [14]:
# 每个参数只能传一次，不能使用两种方法重复传
remainder(20, number=20)

TypeError: remainder() got multiple values for argument 'number'

将'**'加在{}前，会将键值对以关键字形式传入

In [19]:
my_kwargs = {'number': 20,
             'divisor': 7}

In [20]:
assert remainder(**my_kwargs) ==6

In [22]:
#也可以将dict与关键字结合使用
my_kwargs1 = {'divisor': 7}
assert  remainder(**my_kwargs1,number= 20) ==6

In [25]:
# 或者使用两个字典同事传入
my_kwargs1 = {'number': 20}
my_kwargs2 = {'divisor': 7}
assert  remainder(**my_kwargs1, **my_kwargs2) ==6

要函数接受任意数量的关键字参数，可以使用万能形参，该参数会把传入的参数收集到字典中等待进一步处理

In [1]:
def print_paramaters(**kwargs):
    for k, v in list(kwargs.items()):
        print(f'字典key{k}对应的值为{v}')

In [4]:
print_paramaters(apple = 15, huawei = 'p60')

字典keyapple对应的值为15
字典keyhuawei对应的值为p60


### 使用关键字传参’**kwargs‘的优点
1. 让初次阅读代码的人能够快速理解
2. 可以设定默认值，减少代码量
3. 能够灵活的扩充函数的参数且不影响原有调用

In [5]:
# 2 默认值示例代码
def flow_rate(weight_diff, time_diff):
    return weight_diff/time_diff
weight_diff = 0.5
time_diff = 3
flow = flow_rate(weight_diff, time_diff)
print(f'{flow: .3}kg per secpnd')

 0.167kg per secpnd


In [8]:
# 当希望结果的单位时间不是秒时，需要加入一个参数协调单位
def flow_rate(weight_diff, time_diff, period):
    return (weight_diff/time_diff) * period

In [10]:
# 如此情况，即使是以秒为单位也需要指定period为1
flow_per_second = flow_rate(weight_diff, time_diff, 1)
flow_per_second

0.16666666666666666

In [11]:
# 通过指定默认值后可以省略
def flow_rate(weight_diff, time_diff, period = 1):
    return (weight_diff/time_diff) * period

In [13]:
flow_per_second = flow_rate(weight_diff, time_diff, 1)
flow_per_hours = flow_rate(weight_diff, time_diff, 3600)
print(f'一秒的流量为{flow_per_second}, 一小时的流量为{flow_per_hours}')

一秒的流量为0.16666666666666666, 一小时的流量为600.0


In [15]:
# 3 扩充参数
def flow_rate(weight_diff, time_diff, period = 1, units_per_kg = 1):
    return (weight_diff * units_per_kg )/ time_diff * period

In [18]:
pounds_per_hours = flow_rate(weight_diff, time_diff, period=3600,units_per_kg= 2.2)
pounds_per_hours

1320.0

## 25 用None和docstring来面熟默认值会变得参数
如参数是一个可变的值，其默认值在程序初始化时就已经确定，第2、3及以后的调用并不会实时更新

In [41]:
from time import sleep
from datetime import datetime
def log(message, when = datetime.now()):
    print(when)
    print(f'{when} : {message}')
log('Hi there')
sleep(0.1)
log('Second Hi')

2023-08-30 22:19:51.449514
2023-08-30 22:19:51.449514 : Hi there
2023-08-30 22:19:51.449514
2023-08-30 22:19:51.449514 : Second Hi


In [39]:
# 想要实现实时变化的方式，需要将其赋为None，并在参数说明写清运作过程
def log(message, when = None):
    """ 结合时间戳记录log
    
    :param message: 需要打印的信息
    :param when: 当信息被打印时的时间戳，默认为当前时间
    """
    print(when)
    if when is None:
        when = datetime.now()
    print(f'{when} : {message}')

In [40]:
log('Hi there')
sleep(0.1)
log('Second Hi')

None
2023-08-30 22:19:39.920562 : Hi there
None
2023-08-30 22:19:40.022930 : Second Hi


In [34]:
import json
def decode(data, default = {}):
    try:
        return json.loads(data)
    except ValueError:
        print('非json数据')
        return default
foo = decode('bad data')
foo['stuff'] = 5
print('Foo:', foo)
bar = decode('asls bad')
bar['meep'] = 1
assert bar == foo
print('Bar:', bar)
# 因为字典已经初始化了，所以每个值都要传进了同一个字典

非json数据
Foo: {'stuff': 5}
非json数据
Bar: {'stuff': 5, 'meep': 1}


In [38]:
import json
def decode(data, default = None):
    print(default)
    try:
        return json.loads(data)
    except ValueError:   
        default = {} if default is None else default
        print(f'非json数据,default = {default}')
        return default
foo = decode('bad data')
foo['stuff'] = 5
print('Foo:', foo)
bar = decode('asls bad')
bar['meep'] = 1
# assert bar == foo
print('Bar:', bar)

None
非json数据,default = {}
Foo: {'stuff': 5}
None
非json数据,default = {}
Bar: {'meep': 1}


In [46]:
# 结合文档注释进一步完善代码
from typing import Optional
def log(message, when: Optional[str|datetime] = None):
    """ 结合时间戳记录log
    
    :param message: 需要打印的信息
    :param when: 当信息被打印时的时间戳，默认为当前时间
    """
    if when is None:
        when = datetime.now()
    print(f'{when} : {message}')

In [47]:
log('Hi there')
sleep(0.1)
log('Second Hi')

2023-08-30 22:26:18.681554 : Hi there
2023-08-30 22:26:18.781738 : Second Hi


# 25. 用只能以关键字指定和只能安位置传入的参数来设计清晰的参数列表
当关键字传参与位置传参混合使用时，输入不对会报错，合理地分配参数的属性，能够简化代码同时还可以为后期维护节省时间

In [48]:
def safe_division(number, divisor,
                  ignore_flow, ignore_zero_division):
    try:
        return number/divisor
    except OverflowError:
        if ignore_flow:
            return 0
        else:
            raise 
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise 

In [51]:
safe_division(1.0, 0, False, True)

inf

In [53]:
safe_division(1.0, 10**500, True, False)

0

In [54]:
# 设置默认值简化调用时的传递
def safe_division(number, divisor,
                  ignore_flow = False, ignore_zero_division = False):
    try:
        return number/divisor
    except OverflowError:
        if ignore_flow:
            return 0
        else:
            raise 
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise 

In [0]:
result = safe_division(1.0, 10**500, ignore_flow=True)
result

In [59]:
# 但是此种方式还是无法限定调用者必须使用关键字传参
result = safe_division(1.0, 10**500, True)
result

0

In [61]:
# 声明参数类型可以完成相应限定,'*'后为关键字，前为位置
def safe_division(number, divisor,*,
                  ignore_flow = False, ignore_zero_division = False):
    try:
        return number/divisor
    except OverflowError:
        if ignore_flow:
            return 0
        else:
            raise 
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise 

In [63]:
# 此时不指定关键字会报错
result = safe_division(1.0, 10**500, True)

TypeError: safe_division() takes 2 positional arguments but 3 were given

In [64]:
#前面的'number'/'divisor'仍支持位置+关键字，这样如果调用时使用了关键字传参，会影响前面代码，同时这两个参数也不是函数的接口 
def safe_division(number, divisor,/,*,
                  ignore_flow = False, ignore_zero_division = False):
    try:
        return number/divisor
    except OverflowError:
        if ignore_flow:
            return 0
        else:
            raise 
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise 

In [65]:
#此时前面两个参数只能使用位置传参，此特性py3.8才支持
safe_division(number =1, divisor = 2)

TypeError: safe_division() got some positional-only arguments passed as keyword arguments: 'number, divisor'

In [71]:
# 在’/‘和‘*‘间的参数既支持位置又支持关键字
def safe_division(number, divisor,/,ndigits = 2,*,
                  ignore_flow = False, ignore_zero_division = False):
    try:
        return round(number/divisor, ndigits)
    except OverflowError:
        if ignore_flow:
            return 0
        else:
            raise 
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise 

In [74]:
result = safe_division(22,7)
print(result)
result = safe_division(22,7,5)
print(result)
result = safe_division(22,7,ndigits=3)
print(result)

3.14
3.14286
3.143
